In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
resp_heroes_stats = requests.get(f'https://api.opendota.com/api/heroStats').json()

<h2>Making stats for one hero (anti-mage)

Required columns (stats for players which rank is at least Legend)
- hero_id
- name
- roles
- 5_pick
- 5_win
- 6_pick
- 6_win
- 7_pick
- 7_win
- 8_pick
- 8_win

<h3>Matching ranks to columns</h3>

5_pick - **Legend Rank**<br>
6_pick - **Ancient Rank**<br>
7_pick - **Divine Rank**<br>
8_pick - **Immortal Rank**<br>

In [3]:
columns = ['hero_id', 'name', 'roles', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick', '8_win']

In [4]:
resp_heroes_stats[0]['roles'] = '-'.join(resp_heroes_stats[0]['roles'])
resp_heroes_stats[0]['roles']

'Carry-Escape-Nuker'

In [5]:
dataframe = pd.DataFrame([resp_heroes_stats[0]], columns=columns).set_index('hero_id')
dataframe.head()

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win
hero_id,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24656,12395,10486,5317,4481,2250,1220,633


In [6]:
winrate_columns = columns[3:]
np.partition(winrate_columns, (2, 2))

array(['5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick',
       '8_win'], dtype='<U6')

In [7]:
for column in winrate_columns:
    if column.endswith('pick'):
        column_pick = dataframe[column]
        column_win = dataframe[column.split('_')[0]+'_win']
        winrate = column_win / column_pick
        dataframe[column.split('_')[0]+'_winrate'] = winrate

In [8]:
dataframe

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,5_winrate,6_winrate,7_winrate,8_winrate
hero_id,,,,,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24656,12395,10486,5317,4481,2250,1220,633,0.502717,0.507057,0.50212,0.518852


In [19]:
resp_heroes_stats = requests.get(f'https://api.opendota.com/api/heroStats').json()

<h2>Make winrate stats for all Dota 2 heroes

In [20]:
columns = ['hero_id', 'name', 'roles', '5_pick', '5_win', '6_pick', '6_win', '7_pick', '7_win', '8_pick', '8_win']
winrate_columns = columns[3:]

common_dataframe = pd.DataFrame()
for hero in resp_heroes_stats:
    hero_ = hero.copy()
    hero_['roles'] = '-'.join(hero_['roles'])
    dataframe = pd.DataFrame([hero_], columns=columns).set_index('hero_id')
    for column in winrate_columns:
        if column.endswith('pick'):
            column_pick = dataframe[column]
            column_win = dataframe[column.split('_')[0]+'_win']
            winrate = column_win / column_pick
            dataframe[column.split('_')[0]+'_winrate'] = winrate
    common_dataframe = common_dataframe.append(dataframe)

In [21]:
# mean values for all rows
common_dataframe['mean_winrate'] = common_dataframe[['5_winrate', '6_winrate', '7_winrate', '8_winrate']].mean(axis=1)

In [22]:
common_dataframe

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,5_winrate,6_winrate,7_winrate,8_winrate,mean_winrate
hero_id,,,,,,,,,,,,,,,
1,npc_dota_hero_antimage,Carry-Escape-Nuker,24656,12395,10486,5317,4481,2250,1220,633,0.502717,0.507057,0.502120,0.518852,0.507687
2,npc_dota_hero_axe,Initiator-Durable-Disabler-Jungler-Carry,18640,9302,7279,3551,2658,1275,540,251,0.499034,0.487842,0.479684,0.464815,0.482844
3,npc_dota_hero_bane,Support-Disabler-Nuker-Durable,19823,10418,10671,5732,5915,3209,3038,1654,0.525551,0.537157,0.542519,0.544437,0.537416
4,npc_dota_hero_bloodseeker,Carry-Disabler-Jungler-Nuker-Initiator,9148,4615,4180,2113,1832,916,789,399,0.504482,0.505502,0.500000,0.505703,0.503922
5,npc_dota_hero_crystal_maiden,Support-Disabler-Nuker-Jungler,18859,9236,7670,3758,2925,1362,795,380,0.489740,0.489961,0.465641,0.477987,0.480832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,npc_dota_hero_void_spirit,Carry-Escape-Nuker-Disabler,35216,17797,17808,8949,8611,4313,3440,1711,0.505367,0.502527,0.500871,0.497384,0.501537
128,npc_dota_hero_snapfire,Support-Nuker-Disabler-Escape,22704,10879,10724,5061,5154,2485,2163,1036,0.479167,0.471932,0.482150,0.478964,0.478053
129,npc_dota_hero_mars,Carry-Initiator-Disabler-Durable,29336,13571,14899,6937,7228,3328,3096,1515,0.462606,0.465602,0.460432,0.489341,0.469495


In [12]:
common_dataframe.loc[common_dataframe['mean_winrate'] == common_dataframe['mean_winrate'].min()]

,name,roles,5_pick,5_win,6_pick,6_win,7_pick,7_win,8_pick,8_win,5_winrate,6_winrate,7_winrate,8_winrate,mean_winrate
hero_id,,,,,,,,,,,,,,,
53,npc_dota_hero_furion,Carry-Jungler-Pusher-Escape-Nuker,10034,4234,4314,1845,1840,808,863,400,0.421965,0.427677,0.43913,0.463499,0.438068


In [13]:
common_dataframe['mean_winrate'].min()

0.4380681257379716

In [14]:
common_dataframe.to_csv('heroes_winrate.csv')

<h2>Heroes matchups

In [15]:
# /heroes/{hero_id}/matchups
resp_heroes_matchups = requests.get(f'https://api.opendota.com/api/heroStats').json()